In [3]:
from __future__ import print_function

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn

In [5]:
import torchvision
import torchvision.transforms as transforms

In [6]:
import os
import sys
import time
import argparse
import numpy as np

In [7]:
from torch.autograd import Variable

In [8]:
lr = 0.001
dataset = 'mnist'
start_epoch = 1
num_epochs = 10
batch_size = 256
optim_type = 'SGD'

In [9]:
use_cuda = torch.cuda.is_available()

In [10]:
best_acc = 0
resize=32

In [11]:
# Data Uplaod
print('\n[Phase 1] : Data Preparation')

transform_train = transforms.Compose([
    transforms.Resize((resize, resize)),
    transforms.ToTensor(),
]) 

transform_test = transforms.Compose([
    transforms.Resize((resize, resize)),
    transforms.ToTensor(),
])


[Phase 1] : Data Preparation


In [28]:
if (dataset == 'mnist'):
    print("| Preparing MNIST dataset...")
    trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_train)
    testset = torchvision.datasets.MNIST(root='./data', train=False, download=False, transform=transform_test)
    outputs = 10
    inputs = 1
    
elif(dataset == 'cifar10'):
    print("| Preparing CIFAR-10 dataset...")
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=False, transform=transform_test)
    num_classes = 10
    inputs=3

| Preparing CIFAR-10 dataset...


In [29]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)

In [15]:

def conv_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        #nn.init.xavier_uniform(m.weight, gain=np.sqrt(2))
        nn.init.normal_(m.weight, mean=0, std=1)
        nn.init.constant(m.bias, 0)

class AlexNet(nn.Module):

    def __init__(self, num_classes, inputs=3):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(inputs, 64, kernel_size=11, stride=4, padding=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [30]:
net = AlexNet(num_classes = 10,inputs=3)
file_name = 'alexnet-'

In [31]:
if use_cuda:
    net.cuda()
    net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True

In [32]:
criterion = nn.CrossEntropyLoss()

In [33]:
def train(epoch):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=5e-4)

    print('\n=> Training Epoch #%d, LR=%.4f' %(epoch, lr))
    for batch_idx, (inputs_value, targets) in enumerate(trainloader):
        if use_cuda:
            inputs_value, targets = inputs_value.cuda(), targets.cuda() # GPU settings
        optimizer.zero_grad()
        inputs_value, targets = Variable(inputs_value), Variable(targets)
        outputs = net(inputs_value)               # Forward Propagation
        loss = criterion(outputs, targets)  # Loss
        loss.backward()  # Backward Propagation
        optimizer.step() # Optimizer update

        train_loss += loss.data[0]
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        print ('| Epoch [%3d/%3d] Iter[%3d/%3d]\t\tLoss: %.4f Acc@1: %.3f%%'
                %(epoch, num_epochs, batch_idx+1,
                    (len(trainset)//batch_size)+1, loss.data[0], 100.*correct/total))

In [34]:
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs_value, targets) in enumerate(testloader):
        if use_cuda:
            inputs_value, targets = inputs_value.cuda(), targets.cuda()
        with torch.no_grad():
            inputs_value, targets = Variable(inputs_value), Variable(targets)
        outputs = net(inputs_value)
        loss = criterion(outputs, targets)

        test_loss += loss.data[0]
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

    # Save checkpoint when best model
    acc = 100.*correct/total
    print("\n| Validation Epoch #%d\t\t\tLoss: %.4f Acc@1: %.2f%%" %(epoch, loss.data[0], acc))
    
    if acc > best_acc:
        best_acc = acc

In [35]:
def get_hms(seconds):
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)

    return h, m, s

In [22]:
elapsed_time = 0
for epoch in range(start_epoch, start_epoch+num_epochs):
    start_time = time.time()

    train(epoch)
    test(epoch)

    epoch_time = time.time() - start_time
    elapsed_time += epoch_time
    print('| Elapsed time : %d:%02d:%02d'  %(get_hms(elapsed_time)))


=> Training Epoch #1, LR=0.0010


/home/insideout/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/insideout/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


| Epoch [  1/ 10] 		Loss: 2.3043 Acc@1: 7.000%
| Epoch [  1/ 10] 		Loss: 2.3030 Acc@1: 7.000%
| Epoch [  1/ 10] 		Loss: 2.3016 Acc@1: 8.000%
| Epoch [  1/ 10] 		Loss: 2.2975 Acc@1: 8.000%
| Epoch [  1/ 10] 		Loss: 2.2847 Acc@1: 9.000%
| Epoch [  1/ 10] 		Loss: 2.2920 Acc@1: 9.000%
| Epoch [  1/ 10] 		Loss: 2.2829 Acc@1: 9.000%
| Epoch [  1/ 10] 		Loss: 2.2713 Acc@1: 10.000%
| Epoch [  1/ 10] 		Loss: 2.2392 Acc@1: 11.000%
| Epoch [  1/ 10] 		Loss: 2.2213 Acc@1: 11.000%
| Epoch [  1/ 10] 		Loss: 2.1893 Acc@1: 12.000%
| Epoch [  1/ 10] 		Loss: 2.1104 Acc@1: 14.000%
| Epoch [  1/ 10] 		Loss: 2.0971 Acc@1: 14.000%
| Epoch [  1/ 10] 		Loss: 1.9626 Acc@1: 15.000%
| Epoch [  1/ 10] 		Loss: 1.8962 Acc@1: 15.000%
| Epoch [  1/ 10] 		Loss: 1.8060 Acc@1: 17.000%
| Epoch [  1/ 10] 		Loss: 1.7210 Acc@1: 18.000%
| Epoch [  1/ 10] 		Loss: 1.7966 Acc@1: 19.000%
| Epoch [  1/ 10] 		Loss: 1.6712 Acc@1: 20.000%
| Epoch [  1/ 10] 		Loss: 1.8403 Acc@1: 20.000%
| Epoch [  1/ 10] 		Loss: 1.7120 Acc@1: 21.000%

| Epoch [  1/ 10] 		Loss: 0.2023 Acc@1: 75.000%
| Epoch [  1/ 10] 		Loss: 0.1291 Acc@1: 75.000%
| Epoch [  1/ 10] 		Loss: 0.0772 Acc@1: 75.000%
| Epoch [  1/ 10] 		Loss: 0.1750 Acc@1: 76.000%
| Epoch [  1/ 10] 		Loss: 0.1320 Acc@1: 76.000%
| Epoch [  1/ 10] 		Loss: 0.1440 Acc@1: 76.000%
| Epoch [  1/ 10] 		Loss: 0.1370 Acc@1: 76.000%
| Epoch [  1/ 10] 		Loss: 0.0787 Acc@1: 76.000%
| Epoch [  1/ 10] 		Loss: 0.1512 Acc@1: 76.000%
| Epoch [  1/ 10] 		Loss: 0.1140 Acc@1: 76.000%
| Epoch [  1/ 10] 		Loss: 0.0659 Acc@1: 76.000%
| Epoch [  1/ 10] 		Loss: 0.1423 Acc@1: 76.000%
| Epoch [  1/ 10] 		Loss: 0.1239 Acc@1: 77.000%
| Epoch [  1/ 10] 		Loss: 0.2332 Acc@1: 77.000%
| Epoch [  1/ 10] 		Loss: 0.1126 Acc@1: 77.000%
| Epoch [  1/ 10] 		Loss: 0.1716 Acc@1: 77.000%
| Epoch [  1/ 10] 		Loss: 0.1803 Acc@1: 77.000%
| Epoch [  1/ 10] 		Loss: 0.1532 Acc@1: 77.000%
| Epoch [  1/ 10] 		Loss: 0.1314 Acc@1: 77.000%
| Epoch [  1/ 10] 		Loss: 0.1018 Acc@1: 77.000%
| Epoch [  1/ 10] 		Loss: 0.1112 Acc@1: 

/home/insideout/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  from ipykernel import kernelapp as app


...

| Epoch [  7/ 10] 		Loss: 0.0344 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0141 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0174 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0784 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0631 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0433 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0218 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0494 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0517 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0233 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0701 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0576 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0317 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0129 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0491 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0529 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0079 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0256 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0136 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0322 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0163 Acc@1: 

| Epoch [  8/ 10] 		Loss: 0.0264 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0101 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0167 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0132 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0125 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0189 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0153 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0341 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0314 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0082 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0080 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0529 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0342 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0110 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0089 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0330 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0490 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0330 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0613 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0176 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0116 Acc@1: 

| Epoch [  9/ 10] 		Loss: 0.0257 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0328 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0236 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0095 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0201 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0374 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0161 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0075 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0212 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0241 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0348 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0267 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0269 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0105 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0112 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0313 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0316 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0149 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0429 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0279 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0209 Acc@1: 

| Epoch [ 10/ 10] 		Loss: 0.0056 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0577 Acc@1: 98.000%
| Epoch [ 10/ 10] 		Loss: 0.0332 Acc@1: 98.000%
| Epoch [ 10/ 10] 		Loss: 0.0136 Acc@1: 98.000%
| Epoch [ 10/ 10] 		Loss: 0.0362 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0173 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0088 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0175 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0074 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0169 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0458 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0470 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0174 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0262 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0433 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0105 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0099 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0088 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0295 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0261 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0075 Acc@1: 

| Epoch [ 10/ 10] 		Loss: 0.0319 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0231 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0265 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0712 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0540 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0422 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0171 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0143 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0389 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0338 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0586 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0275 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0142 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0252 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0357 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0285 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0144 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0116 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0261 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0297 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0475 Acc@1: 

In [23]:
print('\n[Phase 4] : Testing model')
print('* Test results : Acc@1 = %.2f%%' %(best_acc))


[Phase 4] : Testing model
* Test results : Acc@1 = 99.00%


In [36]:
dataset = 'cifar10'

In [37]:
elapsed_time = 0
for epoch in range(start_epoch, start_epoch+num_epochs):
    start_time = time.time()

    train(epoch)
    test(epoch)

    epoch_time = time.time() - start_time
    elapsed_time += epoch_time
    print('| Elapsed time : %d:%02d:%02d'  %(get_hms(elapsed_time)))


=> Training Epoch #1, LR=0.0010


/home/insideout/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/insideout/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


| Epoch [  1/ 10] Iter[  1/196]		Loss: 2.3068 Acc@1: 10.000%
| Epoch [  1/ 10] Iter[  2/196]		Loss: 2.3009 Acc@1: 9.000%
| Epoch [  1/ 10] Iter[  3/196]		Loss: 2.3046 Acc@1: 9.000%
| Epoch [  1/ 10] Iter[  4/196]		Loss: 2.3019 Acc@1: 10.000%
| Epoch [  1/ 10] Iter[  5/196]		Loss: 2.3015 Acc@1: 9.000%
| Epoch [  1/ 10] Iter[  6/196]		Loss: 2.2963 Acc@1: 10.000%
| Epoch [  1/ 10] Iter[  7/196]		Loss: 2.2714 Acc@1: 11.000%
| Epoch [  1/ 10] Iter[  8/196]		Loss: 2.2950 Acc@1: 11.000%
| Epoch [  1/ 10] Iter[  9/196]		Loss: 2.2902 Acc@1: 10.000%
| Epoch [  1/ 10] Iter[ 10/196]		Loss: 2.2923 Acc@1: 11.000%
| Epoch [  1/ 10] Iter[ 11/196]		Loss: 2.2942 Acc@1: 11.000%
| Epoch [  1/ 10] Iter[ 12/196]		Loss: 2.2873 Acc@1: 11.000%
| Epoch [  1/ 10] Iter[ 13/196]		Loss: 2.2822 Acc@1: 11.000%
| Epoch [  1/ 10] Iter[ 14/196]		Loss: 2.2887 Acc@1: 11.000%
| Epoch [  1/ 10] Iter[ 15/196]		Loss: 2.2524 Acc@1: 11.000%
| Epoch [  1/ 10] Iter[ 16/196]		Loss: 2.2419 Acc@1: 11.000%
| Epoch [  1/ 10] Iter[ 17/

| Epoch [  1/ 10] Iter[140/196]		Loss: 2.0377 Acc@1: 15.000%
| Epoch [  1/ 10] Iter[141/196]		Loss: 1.9812 Acc@1: 15.000%
| Epoch [  1/ 10] Iter[142/196]		Loss: 2.0429 Acc@1: 16.000%
| Epoch [  1/ 10] Iter[143/196]		Loss: 1.9538 Acc@1: 16.000%
| Epoch [  1/ 10] Iter[144/196]		Loss: 1.9966 Acc@1: 16.000%
| Epoch [  1/ 10] Iter[145/196]		Loss: 1.9693 Acc@1: 16.000%
| Epoch [  1/ 10] Iter[146/196]		Loss: 1.9742 Acc@1: 16.000%
| Epoch [  1/ 10] Iter[147/196]		Loss: 1.9627 Acc@1: 16.000%
| Epoch [  1/ 10] Iter[148/196]		Loss: 1.9797 Acc@1: 16.000%
| Epoch [  1/ 10] Iter[149/196]		Loss: 1.9393 Acc@1: 16.000%
| Epoch [  1/ 10] Iter[150/196]		Loss: 1.9597 Acc@1: 16.000%
| Epoch [  1/ 10] Iter[151/196]		Loss: 1.9843 Acc@1: 16.000%
| Epoch [  1/ 10] Iter[152/196]		Loss: 1.8873 Acc@1: 16.000%
| Epoch [  1/ 10] Iter[153/196]		Loss: 1.8846 Acc@1: 16.000%
| Epoch [  1/ 10] Iter[154/196]		Loss: 1.9329 Acc@1: 16.000%
| Epoch [  1/ 10] Iter[155/196]		Loss: 1.9350 Acc@1: 16.000%
| Epoch [  1/ 10] Iter[1

/home/insideout/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  from ipykernel import kernelapp as app


...

| Epoch [  7/ 10] Iter[114/196]		Loss: 1.0845 Acc@1: 54.000%
| Epoch [  7/ 10] Iter[115/196]		Loss: 1.1752 Acc@1: 54.000%
| Epoch [  7/ 10] Iter[116/196]		Loss: 1.2445 Acc@1: 54.000%
| Epoch [  7/ 10] Iter[117/196]		Loss: 1.1412 Acc@1: 54.000%
| Epoch [  7/ 10] Iter[118/196]		Loss: 1.2850 Acc@1: 54.000%
| Epoch [  7/ 10] Iter[119/196]		Loss: 1.1606 Acc@1: 54.000%
| Epoch [  7/ 10] Iter[120/196]		Loss: 1.2306 Acc@1: 54.000%
| Epoch [  7/ 10] Iter[121/196]		Loss: 1.1270 Acc@1: 54.000%
| Epoch [  7/ 10] Iter[122/196]		Loss: 1.2074 Acc@1: 54.000%
| Epoch [  7/ 10] Iter[123/196]		Loss: 1.2684 Acc@1: 54.000%
| Epoch [  7/ 10] Iter[124/196]		Loss: 1.2308 Acc@1: 54.000%
| Epoch [  7/ 10] Iter[125/196]		Loss: 1.1449 Acc@1: 54.000%
| Epoch [  7/ 10] Iter[126/196]		Loss: 1.2008 Acc@1: 54.000%
| Epoch [  7/ 10] Iter[127/196]		Loss: 1.1228 Acc@1: 54.000%
| Epoch [  7/ 10] Iter[128/196]		Loss: 1.2308 Acc@1: 54.000%
| Epoch [  7/ 10] Iter[129/196]		Loss: 1.1319 Acc@1: 54.000%
| Epoch [  7/ 10] Iter[1

| Epoch [  8/ 10] Iter[ 58/196]		Loss: 1.1348 Acc@1: 55.000%
| Epoch [  8/ 10] Iter[ 59/196]		Loss: 1.0845 Acc@1: 55.000%
| Epoch [  8/ 10] Iter[ 60/196]		Loss: 1.1830 Acc@1: 55.000%
| Epoch [  8/ 10] Iter[ 61/196]		Loss: 1.2744 Acc@1: 55.000%
| Epoch [  8/ 10] Iter[ 62/196]		Loss: 1.1329 Acc@1: 55.000%
| Epoch [  8/ 10] Iter[ 63/196]		Loss: 1.1883 Acc@1: 55.000%
| Epoch [  8/ 10] Iter[ 64/196]		Loss: 1.1675 Acc@1: 55.000%
| Epoch [  8/ 10] Iter[ 65/196]		Loss: 1.1389 Acc@1: 55.000%
| Epoch [  8/ 10] Iter[ 66/196]		Loss: 1.1236 Acc@1: 55.000%
| Epoch [  8/ 10] Iter[ 67/196]		Loss: 1.0927 Acc@1: 55.000%
| Epoch [  8/ 10] Iter[ 68/196]		Loss: 1.2563 Acc@1: 55.000%
| Epoch [  8/ 10] Iter[ 69/196]		Loss: 1.1196 Acc@1: 55.000%
| Epoch [  8/ 10] Iter[ 70/196]		Loss: 1.1244 Acc@1: 55.000%
| Epoch [  8/ 10] Iter[ 71/196]		Loss: 1.1179 Acc@1: 55.000%
| Epoch [  8/ 10] Iter[ 72/196]		Loss: 1.1692 Acc@1: 55.000%
| Epoch [  8/ 10] Iter[ 73/196]		Loss: 1.0600 Acc@1: 55.000%
| Epoch [  8/ 10] Iter[ 


| Validation Epoch #8			Loss: 0.9438 Acc@1: 55.00%
| Elapsed time : 0:00:38

=> Training Epoch #9, LR=0.0010
| Epoch [  9/ 10] Iter[  1/196]		Loss: 1.1043 Acc@1: 58.000%
| Epoch [  9/ 10] Iter[  2/196]		Loss: 2.9547 Acc@1: 42.000%
| Epoch [  9/ 10] Iter[  3/196]		Loss: 1.3814 Acc@1: 47.000%
| Epoch [  9/ 10] Iter[  4/196]		Loss: 1.4605 Acc@1: 47.000%
| Epoch [  9/ 10] Iter[  5/196]		Loss: 1.5788 Acc@1: 47.000%
| Epoch [  9/ 10] Iter[  6/196]		Loss: 1.4268 Acc@1: 48.000%
| Epoch [  9/ 10] Iter[  7/196]		Loss: 1.3580 Acc@1: 49.000%
| Epoch [  9/ 10] Iter[  8/196]		Loss: 1.3253 Acc@1: 49.000%
| Epoch [  9/ 10] Iter[  9/196]		Loss: 1.3101 Acc@1: 49.000%
| Epoch [  9/ 10] Iter[ 10/196]		Loss: 1.3084 Acc@1: 50.000%
| Epoch [  9/ 10] Iter[ 11/196]		Loss: 1.3177 Acc@1: 50.000%
| Epoch [  9/ 10] Iter[ 12/196]		Loss: 1.3616 Acc@1: 50.000%
| Epoch [  9/ 10] Iter[ 13/196]		Loss: 1.4101 Acc@1: 50.000%
| Epoch [  9/ 10] Iter[ 14/196]		Loss: 1.2205 Acc@1: 50.000%
| Epoch [  9/ 10] Iter[ 15/196]		Los

| Epoch [  9/ 10] Iter[146/196]		Loss: 1.0601 Acc@1: 59.000%
| Epoch [  9/ 10] Iter[147/196]		Loss: 1.2749 Acc@1: 59.000%
| Epoch [  9/ 10] Iter[148/196]		Loss: 1.0671 Acc@1: 59.000%
| Epoch [  9/ 10] Iter[149/196]		Loss: 1.1331 Acc@1: 59.000%
| Epoch [  9/ 10] Iter[150/196]		Loss: 1.0948 Acc@1: 59.000%
| Epoch [  9/ 10] Iter[151/196]		Loss: 1.0912 Acc@1: 59.000%
| Epoch [  9/ 10] Iter[152/196]		Loss: 1.0008 Acc@1: 59.000%
| Epoch [  9/ 10] Iter[153/196]		Loss: 1.1050 Acc@1: 59.000%
| Epoch [  9/ 10] Iter[154/196]		Loss: 1.0148 Acc@1: 59.000%
| Epoch [  9/ 10] Iter[155/196]		Loss: 1.0884 Acc@1: 59.000%
| Epoch [  9/ 10] Iter[156/196]		Loss: 1.1661 Acc@1: 59.000%
| Epoch [  9/ 10] Iter[157/196]		Loss: 1.2102 Acc@1: 59.000%
| Epoch [  9/ 10] Iter[158/196]		Loss: 1.0102 Acc@1: 59.000%
| Epoch [  9/ 10] Iter[159/196]		Loss: 1.2452 Acc@1: 59.000%
| Epoch [  9/ 10] Iter[160/196]		Loss: 1.0603 Acc@1: 59.000%
| Epoch [  9/ 10] Iter[161/196]		Loss: 0.9511 Acc@1: 59.000%
| Epoch [  9/ 10] Iter[1

| Epoch [ 10/ 10] Iter[ 91/196]		Loss: 1.1353 Acc@1: 61.000%
| Epoch [ 10/ 10] Iter[ 92/196]		Loss: 1.1131 Acc@1: 60.000%
| Epoch [ 10/ 10] Iter[ 93/196]		Loss: 1.0213 Acc@1: 61.000%
| Epoch [ 10/ 10] Iter[ 94/196]		Loss: 0.9236 Acc@1: 61.000%
| Epoch [ 10/ 10] Iter[ 95/196]		Loss: 0.9735 Acc@1: 61.000%
| Epoch [ 10/ 10] Iter[ 96/196]		Loss: 0.9517 Acc@1: 61.000%
| Epoch [ 10/ 10] Iter[ 97/196]		Loss: 1.0379 Acc@1: 61.000%
| Epoch [ 10/ 10] Iter[ 98/196]		Loss: 1.0367 Acc@1: 61.000%
| Epoch [ 10/ 10] Iter[ 99/196]		Loss: 1.0444 Acc@1: 61.000%
| Epoch [ 10/ 10] Iter[100/196]		Loss: 1.0293 Acc@1: 61.000%
| Epoch [ 10/ 10] Iter[101/196]		Loss: 0.9356 Acc@1: 61.000%
| Epoch [ 10/ 10] Iter[102/196]		Loss: 1.0653 Acc@1: 61.000%
| Epoch [ 10/ 10] Iter[103/196]		Loss: 1.0633 Acc@1: 61.000%
| Epoch [ 10/ 10] Iter[104/196]		Loss: 0.9964 Acc@1: 61.000%
| Epoch [ 10/ 10] Iter[105/196]		Loss: 1.0776 Acc@1: 61.000%
| Epoch [ 10/ 10] Iter[106/196]		Loss: 1.0065 Acc@1: 61.000%
| Epoch [ 10/ 10] Iter[1